In [ ]:
!pip install gym[atari]
!pip install autorom[accept-rom-license]
!pip install autorom[accept-rom-license]
!pip install --upgrade gym==0.24.0
!pip install --upgrade ale_py==0.7.2
!pip install ale_py==0.7.2
!pip install keras-rl2
!pip install --upgrade tensorflow==2.8.0

In [ ]:
import gym
import random
import ale_py
import matplotlib.pyplot as plt
from IPython.display import clear_output
import numpy as np


In [ ]:
env=gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape
actions=env.action_space.n
env.render_mode='human'
disable_env_checker=True

In [ ]:
import cv2

class ImageCompressionWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.compressed_shape = (32, 32, 3)  # Set the desired compressed shape
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=self.compressed_shape, dtype=np.uint8)

    def observation(self, observation):
      #print(observation)
      compressed_observation = self.compress_image(observation)
      return compressed_observation

    def compress_image(self,image):
      compressed_image = cv2.resize(image, (self.compressed_shape[1], self.compressed_shape[0]), interpolation=cv2.INTER_AREA)
      return compressed_image
env = ImageCompressionWrapper(env)
height, width, channels = env.observation_space.shape
print(height,width)

In [ ]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [ ]:
episodes=5
img = plt.imshow(env.render(mode="rgb_array"))
for episode in range(episodes):
    state=env.reset()
    done=False
    score=0
    while not done:
        img.set_data(env.render(mode='rgb_array'))
        display(plt.gcf())
        action=random.choice([0,1,2,3,4,5])
        #print(env.step(action))
        n_state, reward, done,info=env.step(action)
        score+=reward
        clear_output(wait=True)
        #print(score)
    print('Episode:{} Score:{}'.format(episode,score))
env.close()

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [ ]:
def build_model(height,width, channels, actions):
    model=Sequential()
    model.add(Convolution2D(32,(8,8),strides=(4,4),activation='relu',input_shape=(1,height,width,channels)))
    model.add(Convolution2D(32,(4,4),strides=(2,2),activation='relu'))
    #model.add(Convolution2D(64,(3,3),activation='relu'))
    model.add(Flatten())
    model.add(Dense(128,activation='relu'))
    model.add(Dense(64,activation='relu'))
    model.add(Dense(actions,activation='linear'))
    return model


In [ ]:
del model

In [ ]:
model=build_model(height,width,channels,actions)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1, 7, 7, 32)       6176      
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 2, 2, 32)       16416     
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 6)                 390       
                                                                 
Total params: 47,750
Trainable params: 47,750
Non-traina

In [ ]:
from tensorflow import keras
import rl
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy,EpsGreedyQPolicy

In [ ]:
def build_agent(model,actions):
    policy=LinearAnnealedPolicy(EpsGreedyQPolicy(),attr='eps',value_max=1.,value_min=.1, value_test=.2, nb_steps=10000)
    memory=SequentialMemory(limit=1000,window_length=1)
    dqn=DQNAgent(model=model,memory=memory,policy=policy,
                 enable_dueling_network=True, dueling_type='avg',nb_actions=actions,nb_steps_warmup=1000)
    return dqn

In [ ]:
dqn=build_agent(model,actions)
dqn.compile(Adam(learning_rate=1e-4))
dqn.fit(env,nb_steps=10000,visualize=False,verbose=2)

Training for 10000 steps ...
  796/10000: episode: 1, duration: 2.029s, episode steps: 796, steps per second: 392, episode reward: 165.000, mean reward:  0.207 [ 0.000, 30.000], mean action: 2.521 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --


/usr/local/lib/python3.10/dist-packages/gym/utils/seeding.py:63: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high, size, dtype])` instead.
  deprecation(
/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


 1678/10000: episode: 2, duration: 16.287s, episode steps: 882, steps per second:  54, episode reward: 215.000, mean reward:  0.244 [ 0.000, 30.000], mean action: 2.539 [0.000, 5.000],  loss: 1.555089, mean_q: 4.713728, mean_eps: 0.879490


/usr/local/lib/python3.10/dist-packages/gym/utils/seeding.py:63: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high, size, dtype])` instead.
  deprecation(


 3064/10000: episode: 3, duration: 30.790s, episode steps: 1386, steps per second:  45, episode reward: 630.000, mean reward:  0.455 [ 0.000, 200.000], mean action: 2.659 [0.000, 5.000],  loss: 14.960479, mean_q: 5.032790, mean_eps: 0.786655


...

 8922/10000: episode: 11, duration: 9.747s, episode steps: 406, steps per second:  42, episode reward: 55.000, mean reward:  0.135 [ 0.000, 20.000], mean action: 2.436 [0.000, 5.000],  loss: 2.250030, mean_q: 4.317207, mean_eps: 0.215335


/usr/local/lib/python3.10/dist-packages/gym/utils/seeding.py:63: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high, size, dtype])` instead.
  deprecation(


 9809/10000: episode: 12, duration: 19.016s, episode steps: 887, steps per second:  47, episode reward: 200.000, mean reward:  0.225 [ 0.000, 30.000], mean action: 2.807 [0.000, 5.000],  loss: 1.449578, mean_q: 4.691851, mean_eps: 0.157150


/usr/local/lib/python3.10/dist-packages/gym/utils/seeding.py:63: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high, size, dtype])` instead.
  deprecation(


done, took 202.836 seconds
